In [ ]:
#! pip install pyautogen

In [2]:
import autogen
import os

api_key = os.environ['OPENAI_API_KEY']

In [90]:
import random

config_list = [
    {
        "model": "gpt-4",
        "api_key": api_key
    },
#     {
#         "model": "gpt-3.5-turbo-16k",
#         "api_key": api_key
#     },
]

In [36]:
url = "https://docs.cloud.coinbase.com/sign-in-with-coinbase/docs/api-prices"


In [37]:
def create_system_prompt(text: str) -> str:
    DEFAULT_SYSTEM_MESSAGE = f"""You are a helpful AI assistant.
    {text}
    Solve tasks using your coding and language skills.
    In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
        1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
        2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
    Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
    When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
    If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
    If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
    When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
    """
    #Reply "TERMINATE" in the end when everything is done.
    return DEFAULT_SYSTEM_MESSAGE

In [83]:
message = f"""
When ever you receive an URL, scrape the names of all the HTML elements (just the element names, not the content). 
You can use BeautifulSoup to scrape the web.

The script should print all the HTML elements bellow the following headline: "HTML ELEMENT NAMES:"
Output example:
"HTML ELEMENT NAMES:
ul, main, article, pre, body, h4...
"

Your ONLY job is scrape the names of all the HTML elements from the providen URL. 
DON'T implement or do anything else!!
Ignore the question, just scrape the HTML elements from the URL!
"""
# Finally write the information that you found after the headline - "{web_information_headline}"

HTML_ELEMENTS_NAMES_SYSTEM_MESSAGE = create_system_prompt(message)
HTML_ELEMENTS_NAMES_SYSTEM_MESSAGE

'You are a helpful AI assistant.\n    \nWhen ever you receive an URL, scrape the names of all the HTML elements (just the element names, not the content). \nYou can use BeautifulSoup to scrape the web.\n\nThe script should print all the HTML elements bellow the following headline: "HTML ELEMENT NAMES:"\nOutput example:\n"HTML ELEMENT NAMES:\nul, main, article, pre, body, h4...\n"\n\nYour ONLY job is scrape the names of all the HTML elements from the providen URL. \nDON\'T implement or do anything else!!\nIgnore the question, just scrape the HTML elements from the URL!\n\n    Solve tasks using your coding and language skills.\n    In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.\n        1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the op

In [71]:
llm_config={
    "request_timeout": 600,
    "seed": random.randint(100, 900),
    "config_list": config_list,
    "temperature": 0,
}

element_names_scraper = autogen.AssistantAgent(
    name="element_names_scraper",
    llm_config=llm_config,
    system_message=HTML_ELEMENTS_NAMES_SYSTEM_MESSAGE,
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={"work_dir": "web/elements"},
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
)

In [72]:
message = f"""
Explore the coinbase api, you can find the documentation at: {url}.
Finally, show me examples how to get cryptocurrency prices
"""
user_proxy.initiate_chat(
    element_names_scraper,
    message=message,
)

user_proxy (to element_names_scraper):


Explore the coinbase api, you can find the documentation at: https://docs.cloud.coinbase.com/sign-in-with-coinbase/docs/api-prices.
Finally, show me examples how to get cryptocurrency prices


--------------------------------------------------------------------------------
element_names_scraper (to user_proxy):

Sure, I will scrape the HTML elements from the provided URL.

Here is the Python code to do that:

```python
# filename: scrape_html_elements.py

import requests
from bs4 import BeautifulSoup

url = "https://docs.cloud.coinbase.com/sign-in-with-coinbase/docs/api-prices"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

elements = [tag.name for tag in soup.find_all()]

print("HTML ELEMENT NAMES:")
for element in set(elements):
    print(element)
```

This script will send a GET request to the provided URL, parse the HTML response using BeautifulSoup, and then print the names of all the HTML elements found o

KeyboardInterrupt: 

In [82]:
web_information_headline = "INFORMATION FROM THE WEB:"
message = f"""
Scrape all the HTML elements which could contain the information for solving the users problem. 
NEVER scrape the whole page, only the elements that you think are useful. 
Choose a MAXIMUM of 5 HTML elements that seem most relevant to the given problem!!!
NEVER inspect more then 5 elements!!!
You can use BeautifulSoup to scrape the web.
Your ONLY job is to write a script for retrieving information/documentation from the providen URL. 
i.e. only write the code which scrapes wanted HTML elements by using BeautifulSoup. 
DON'T implement or do anything else!!

If the user asks you to implement something else, respond with "I am not in charge of solving this task."
"""
# Finally write the information that you found after the headline - "{web_information_headline}"

HTML_ELEMENTS_SYSTEM_MESSAGE = create_system_prompt(message)
HTML_ELEMENTS_SYSTEM_MESSAGE

'You are a helpful AI assistant.\n    \nScrape all the HTML elements which could contain the information for solving the users problem. \nNEVER scrape the whole page, only the elements that you think are useful. \nChoose a MAXIMUM of 5 HTML elements that seem most relevant to the given problem!!!\nNEVER inspect more then 5 elements!!!\nYou can use BeautifulSoup to scrape the web.\nYour ONLY job is to write a script for retrieving information/documentation from the providen URL. \ni.e. only write the code which scrapes wanted HTML elements by using BeautifulSoup. \nDON\'T implement or do anything else!!\n\nIf the user asks you to implement something else, respond with "I am not in charge of solving this task."\n\n    Solve tasks using your coding and language skills.\n    In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.\n        1. When you need to collect info, use the code to output the info you need

In [62]:
llm_config={
    "request_timeout": 600,
    "seed": random.randint(100, 900),
    "config_list": config_list,
    "temperature": 0,
}

scraper_assistant = autogen.AssistantAgent(
    name="scraper_assistant",
    llm_config=llm_config,
    system_message=HTML_ELEMENTS_SYSTEM_MESSAGE,
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={"work_dir": "web"},
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
)

In [63]:
#message = f"Scrpe the information from {url} which could help me retreive the cryptocurrency prices. Don't write any code, just give me the documentation for the API"

message = f"""
Explore the coinbase api, you can find the documentation at: {url}.
Finally, show me examples how to get cryptocurrency prices
"""
user_proxy.initiate_chat(
    scraper_assistant,
    message=message,
)

user_proxy (to scraper_assistant):


Explore the coinbase api, you can find the documentation at: https://docs.cloud.coinbase.com/sign-in-with-coinbase/docs/api-prices.
Finally, show me examples how to get cryptocurrency prices


--------------------------------------------------------------------------------
scraper_assistant (to user_proxy):

To get the cryptocurrency prices from the Coinbase API, you would typically use the `/prices/{cryptocurrency}/spot` endpoint. However, to scrape the relevant information from the documentation page, we can use BeautifulSoup in Python. 

Here is a Python script that scrapes the relevant HTML elements from the provided URL:

```python
# filename: scrape_coinbase.py

import requests
from bs4 import BeautifulSoup

# URL of the documentation
url = "https://docs.cloud.coinbase.com/sign-in-with-coinbase/docs/api-prices"

# Send a GET request to the URL
response = requests.get(url)

# Parse the response content with BeautifulSoup
soup = BeautifulSoup(re


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
user_proxy (to scraper_assistant):

exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "", line 1, in <module>
    from coinbase.wallet.client import Client
ModuleNotFoundError: No module named 'coinbase'


--------------------------------------------------------------------------------
scraper_assistant (to user_proxy):

I apologize for the oversight. It seems the `coinbase` Python package is not installed in your environment. You can install it using pip, which is a package manager for Python. Here's how you can do it:

```sh
pip install coinbase
```

After running this command, you should be able to import the `Client` class from the `coinbase.wallet.client` module and use it to interact with the Coinbase API.

--------------------------------------------------------------

In [54]:
message = f"""
Your job is to write executable python examples for the providen task.
Add "# filename: examples.py" inside the code block you create.
"""
# In order to do that, use the information from the "{web_information_headline}" section.

CODER_SYSTEM_MESSAGE = create_system_prompt(message)
CODER_SYSTEM_MESSAGE

'You are a helpful AI assistant.\n    \nYour job is to write executable python examples for the providen task.\nAdd "# filename: examples.py" inside the code block you create.\n\n    Solve tasks using your coding and language skills.\n    In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.\n        1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.\n        2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.\n    Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear 

In [97]:
llm_config={
    "request_timeout": 600,
    "seed": random.randint(100, 900),
    "config_list": config_list,
    "temperature": 0,
}

scraper_assistant = autogen.AssistantAgent(
    name="scraper_assistant",
    llm_config=llm_config,
    system_message=HTML_ELEMENTS_SYSTEM_MESSAGE,
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
#     is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={"work_dir": "web/all_steps"},
    llm_config=llm_config,
    system_message="""
Your only job is to execute python code blocks.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
)


user_proxy_save_code = autogen.UserProxyAgent(
    name="user_proxy_save_code",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={"work_dir": "web"},
    llm_config=llm_config,
    system_message="""
Save the code inside the examples.py file.    
Reply TERMINATE once you have saved the code.
"""
)

coder = autogen.AssistantAgent(
    name="coder",
    llm_config=llm_config,
    system_message=CODER_SYSTEM_MESSAGE,
)

element_names_scraper = autogen.AssistantAgent(
    name="element_names_scraper",
    llm_config=llm_config,
    system_message=HTML_ELEMENTS_NAMES_SYSTEM_MESSAGE,
)

CHAT_MANAGER_SYSTEM_MESSAGE = """
You are a Group chat manager.

The FIRST message ALWAYS needs to be sent to the element_names_scraper.

The second step is to crate a script for geting additional information from the web.
This task should be executed by scraper_assistant.

After additional information were retreived, the coder should try to solve the initial task.

If the code for the initial task is successfuly executed, user_proxy_save_code should save the code into a file. 

Each script that is created should be executed by user_proxy. 
i.e. When ever someone writes a code block which starts with "```python", user_proxy must execute that code block.
"""

groupchat = autogen.GroupChat(agents=[user_proxy, element_names_scraper, scraper_assistant, coder, user_proxy_save_code], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [99]:
message_prefix = """
# filename: examples.py;

BEFORE TASK: element_names_scraper needs to creata a script for getting all element names from HTML.
Do not proceed until element_names_scraper creates the script!\n
"""
message = f"""
TASK:
Explore the coinbase api, you can find the documentation at: {url}.
Finally, show me examples how to get cryptocurrency prices
"""
user_proxy.initiate_chat(manager, message=message_prefix+message)

user_proxy (to chat_manager):


# filename: examples.py;

BEFORE TASK: element_names_scraper needs to creata a script for getting all element names from HTML.
Do not proceed until element_names_scraper creates the script!


TASK:
Explore the coinbase api, you can find the documentation at: https://docs.cloud.coinbase.com/sign-in-with-coinbase/docs/api-prices.
Finally, show me examples how to get cryptocurrency prices


--------------------------------------------------------------------------------
element_names_scraper (to chat_manager):

# filename: scrape_elements.py
from bs4 import BeautifulSoup
import requests

def get_html_elements(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    elements = [tag.name for tag in soup.find_all()]
    return elements

url = "https://docs.cloud.coinbase.com/sign-in-with-coinbase/docs/api-prices"
elements = get_html_elements(url)

print("HTML ELEMENT NAMES:")
for element in set(elements):
    print(elem

[autogen.oai.completion: 10-13 06:02:56] {238} INFO - retrying in 10 seconds...
Traceback (most recent call last):
  File "/home/robert/.local/lib/python3.11/site-packages/autogen/oai/completion.py", line 208, in _get_response
    response = openai_completion.create(**config)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/robert/.local/lib/python3.11/site-packages/openai/api_resources/chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/robert/.local/lib/python3.11/site-packages/openai/api_resources/abstract/engine_api_resource.py", line 155, in create
    response, _, api_key = requestor.request(
                           ^^^^^^^^^^^^^^^^^^
  File "/home/robert/.local/lib/python3.11/site-packages/openai/api_requestor.py", line 299, in request
    resp, got_stream = self._interpret_response(result, stream)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home

KeyboardInterrupt: 